# Задача на анализ продукта 
Есть некий продукт «Technology». Существует несколько вариантов подключения продукта пользователем:
- Отдельно, только «Technology»
- В связке с вторым продуктом «Service»: оплачивать единый тариф, включающий в себя оба продукта (будем такое подключение называть – Combo).

Второй вариант подключения считается приоритетным, поскольку в перспективе клиент использует больше услуг, дольше остается пользователем, имеет меньшую вероятность отключиться. Поэтому Combo подключение продается со скидкой таким образом, что это подключение становится даже выгоднее, чем только «Technology». У сотрудников продаж есть цель на подключение именно Combo.
Есть гипотеза, что сотрудники продают Combo клиентам, которым продукт «Service» не нужен. Тем самым несут компании больше расходов на скидках, а также насыщают клиентскую базу продукта «Service» неактивными людьми, фейковыми клиентами.

**Описание данных:**
    

- ***technology*** - Объект с клиентами активными* в продукте «Technology» 
  - date_key_src - Период	
  - client_id_tech - ID Клиента в продукте Technology	
  - combo_ind	- Признак Combo 
  - client_id_serv - ID Клиента в продукте Service	


- ***service*** - Объект с клиентами активными* в продукте «Service»
  - time_key_dt -	Период	
  - client_id_serv -	ID Клиента в продукте 
  - segment -	Сегмент активности клиента	
  - sales_ind -	Признак новых продаж (sales\old)	


- ***service_revenue*** -	Объект с доходами по клиентам по продукту «Service»
  - time_key_dt -	Период	
  - client_id_serv -	ID Клиента в продукте Service	
  - revenue -	Доход в условных единицах	


- ***service_fraud*** - Объект с абонентами «Service» в статусе Фрод
  - time_key_dt -	Период	
  - client_id_serv -	ID Клиента в продукте Service	
  - channel -	Канал подключения Фрода	


**Необходимо ответить на вопросы:** 
- Какую динамику показывают базы клиентов «Technology», «Service» и Combo в частности?
- Как абоненты Combo ведут себя с точки зрения продукта «Service»? Насколько стабильно пользуются и сколько за «Service» платят?
- Динамика Combo Продаж. Есть ли среди абонентов FMC продаж – Фрод? Источник фрода?
- Исследовать качество продаж Combo: 
    - Как подключенные клиенты выживают (остаются в базе клиентов) на 1- 4Месяц, 12Месяц с точки зрения продукта «Service»? С точки зрения «Technology»?


## Предобработка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

pd.options.display.max_columns = 30
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
technology = pd.read_csv('technology_subs.csv') 
service = pd.read_csv('service_subs.csv') 
service_revenue = pd.read_csv('service_revenue.csv') 
service_fraud = pd.read_csv('service_fraud.csv')

In [3]:
# Функция первичного просмотра
def first_view(df, tab_name):
    print(f'Таблица {tab_name}')
    print('--------------------------------------------------')
    df.info()
    print('--------------------------------------------------')
    display(df.head())
    print('--------------------------------------------------')
    print('Количество дубликатов:', df.duplicated().sum())
    print('--------------------------------------------------')
    for col in df.columns:
        if len(df[col].unique())<10:
            print(f'Соотношение в столбце {col} в %')
            print('--------------------------------------------------')
            print(df[col].value_counts(normalize=True))
            print('--------------------------------------------------')
        else:
            if df[col].dtype in ['float', 'int']:
                print('Распределение в столбце', col)
                print('--------------------------------------------------')
                print(df[col].describe())
                print('--------------------------------------------------')

### Таблица technology

In [4]:
first_view(technology, 'technology')

Таблица technology
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000968 entries, 0 to 1000967
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   date_key_src    1000968 non-null  object
 1   client_id_tech  1000968 non-null  object
 2   combo_ind       1000968 non-null  int64 
 3   client_id_serv  570589 non-null   object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
--------------------------------------------------


,date_key_src,client_id_tech,combo_ind,client_id_serv
0,2022-02-01,A694897209,1,A64955227227421
1,2021-06-01,A694898225,1,A64900848624998
2,2021-07-01,A694900227,1,A68699999951359
3,2021-10-01,A694902219,1,A64955426727513
4,2022-05-01,A694903225,0,NaN


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Соотношение в столбце combo_ind в %
--------------------------------------------------
1   0.60
0   0.40
Name: combo_ind, dtype: float64
--------------------------------------------------


In [5]:
# Изменим названия столбцов и тип данных в date_key_src, чтобы было удобнее работать с таблицей
technology.columns = ['dt', 'id_tech', 'combo', 'id_serv']
technology.dt = pd.to_datetime(technology.dt, format='%Y-%m-%d')

In [6]:
# Даты
min_date = technology.dt.min()
max_date = technology.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print(f'Изучаем данные за период {len(dates)} дней')

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-07-01 00:00:00
Изучаем данные за период 547 дней


Изучаемый период 547 дней (19 месяцев)

In [7]:
# Посмотрим на пропуски в столбце id_serv
display(technology[technology.id_serv.isna()].sample(5))

print('Соотношение в столбце combo в %, для строк где id_serv=NaN')
display(technology[technology.id_serv.isna()].combo.value_counts(normalize=True))

cnt_comboid_nan = len(technology[(technology.id_serv.isna()) & (technology.combo==1)])
print(f"Количество комбо подключений без id_serv: {cnt_comboid_nan}")  

,dt,id_tech,combo,id_serv
68156,2021-01-01,K6978675140,0,NaN
463483,2021-01-01,V73148542123,0,NaN
707562,2021-04-01,S70017222147,0,NaN
788568,2021-06-01,T73070140941,0,NaN
43895,2022-01-01,I69928812873,0,NaN


Соотношение в столбце combo в %, для строк где id_serv=NaN


0   0.94
1   0.06
Name: combo, dtype: float64

Количество комбо подключений без id_serv: 26727


Для большинства пропусков характерно "соло" подключение, для остальных по каким-то причинам не подгрузились id_serv.
Проверим есть ли дубли в значениях столбца id_tech.

In [8]:
# Дубли
print('Количество уникальных id_tech:', len(technology.id_tech.unique()))
print('Количество не уникальных id_tech:', len(technology)-len(technology.id_tech.unique()))

# ID дублей
id_tech_dub = technology[technology.id_tech.duplicated()].id_tech

# Фильтруем датафрейм
technology_dub = (technology[technology.id_tech.isin(id_tech_dub)]
                       .sort_values(by=['id_tech', 'dt'])).reset_index(drop=True)
technology_dub.head(20)

Количество уникальных id_tech: 989045
Количество не уникальных id_tech: 11923


,dt,id_tech,combo,id_serv
0,2021-03-01,A694988668,1,NaN
1,2021-11-01,A694988668,1,A66937424337215
2,2021-03-01,A694996711,1,NaN
3,2021-11-01,A694996711,1,A64668633012710
4,2021-03-01,A6950721139,0,NaN
5,2021-05-01,A6950721139,1,A68689531250603
6,2022-01-01,A6979885525,1,A6452499713760
7,2022-01-01,A6979885525,1,A6452499713761
8,2022-04-01,A6983657316,1,A68761534056692
9,2022-06-01,A6983657316,0,NaN


Наличие не уникальных id_tech говорит о том, что в базе данных возможно хранится информация о переподключении продукта (напр по завершении договора) или про переход (напр от «Technology» к Combo и наоборот). *Я бы уточнял информацию по струтктуре базы данных и уже на этом основании отсеивал строки.*  

Сценарий перехода возможен для пар строк id_tech, где в одном случае признак комбо есть, а во втором нет. А для тех, где в обоих случаях combo=1, пропуски в id_serv можно заполнить, используя значения для одинакового id_tech. 

In [9]:
# Для каждого id находим количество повторов и среднее значение в столбце combo
id_combo1 = technology_dub.groupby('id_tech', as_index=False).agg({'combo':'mean', 'dt':'count'})

# Делим количество строк на среднее комбо
id_combo1['sign'] = id_combo1['dt']/id_combo1['combo']

# Убираем лишние строки, нам нужны только те, где значение признака = 2
id_combo1 = id_combo1[id_combo1.sign==2].id_tech

# Составим словарь для замены пропущенных значений
dict_replace = technology_dub[technology_dub.id_tech.isin(id_combo1)][['id_tech', 'id_serv']].dropna().drop_duplicates()

# Заменим значения в пропущенных строках на значения из словаря
technology = technology.fillna(dict_replace)
technology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000968 entries, 0 to 1000967
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   dt       1000968 non-null  datetime64[ns]
 1   id_tech  1000968 non-null  object        
 2   combo    1000968 non-null  int64         
 3   id_serv  573930 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 30.5+ MB


Обработали более 3000 пропущеных значений в столбце id_serv.

### Таблица service

In [10]:
first_view(service, 'service_subs')

Таблица service_subs
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1569621 entries, 0 to 1569620
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   time_key_dt     1569621 non-null  object
 1   client_id_serv  1569621 non-null  object
 2   segment         1569621 non-null  object
 3   sales_ind       1569621 non-null  object
dtypes: object(4)
memory usage: 47.9+ MB
--------------------------------------------------


,time_key_dt,client_id_serv,segment,sales_ind
0,2021-01-01,A64309659750,1,old
1,2021-01-01,A643510019114,1,old
2,2021-01-01,A643511179115,1,old
3,2021-01-01,A644422566295,1,old
4,2021-01-01,A644424051299,1,old


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Соотношение в столбце segment в %
--------------------------------------------------
1     0.74
2     0.20
New   0.05
Name: segment, dtype: float64
--------------------------------------------------
Соотношение в столбце sales_ind в %
--------------------------------------------------
old     0.97
sales   0.03
Name: sales_ind, dtype: float64
--------------------------------------------------


In [11]:
# Изменим названия столбцов и типы данных, чтобы было удобнее работать с таблицей
service.columns = ['dt', 'id_serv', 'segment', 'sales']

In [12]:
service.loc[service.segment=='New', 'segment'] = '3'
service.segment = service.segment.astype(int)

In [13]:
service['dt'] = pd.to_datetime(service['dt'], format='%Y-%m-%d')

In [14]:
# Дата
min_date = service.dt.min()
max_date = service.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print(f'Изучаем данные за период {len(dates)} дней')

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-08-01 00:00:00
Изучаем данные за период 578 дней


Изучаемый период на 1 месяц больше, чем для таблицы technology (20 месяцев). *При сравнении с другими вариантами подключения не забыть обрезать данные за последний месяц.*

### Таблица service_revenue

In [15]:
first_view(service_revenue, 'service_revenue')

Таблица service_revenue
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1569579 entries, 0 to 1569578
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   time_key_dt     1569579 non-null  object 
 1   client_id_serv  1569579 non-null  object 
 2   revenue         1569579 non-null  float64
dtypes: float64(1), object(2)
memory usage: 35.9+ MB
--------------------------------------------------


,time_key_dt,client_id_serv,revenue
0,2021-04-01,V644328120231,382.72
1,2021-04-01,K69097158072550,377.43
2,2021-04-01,S65849265234609,682.29
3,2021-04-01,U68598416040936,60.96
4,2021-04-01,S68677155648563,434.01


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Распределение в столбце revenue
--------------------------------------------------
count   1,569,579.00
mean          318.44
std           282.74
min        -3,992.84
25%           128.93
50%           280.69
75%           434.60
max        26,718.27
Name: revenue, dtype: float64
--------------------------------------------------


In [16]:
# Изменим названия столбцов и тип данных, чтобы было удобнее работать с таблицей
service_revenue.columns = ['dt', 'id_serv', 'revenue']
service_revenue.dt = pd.to_datetime(service_revenue.dt, format='%Y-%m-%d')

In [17]:
# Дата
min_date = service_revenue.dt.min()
max_date = service_revenue.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print(f'Изучаем данные за период {len(dates)} дней')

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-08-01 00:00:00
Изучаем данные за период 578 дней


Период точно такой же как и в таблице service (20 месяцев).

В столбце revenue присутствуют отрицательные значения.

### Таблица service_fraud

In [18]:
first_view(service_fraud, 'service_fraud')

Таблица service_fraud
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700 entries, 0 to 2699
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   time_key_dt     2700 non-null   object
 1   client_id_serv  2700 non-null   object
 2   channel         2700 non-null   object
dtypes: object(3)
memory usage: 63.4+ KB
--------------------------------------------------


,time_key_dt,client_id_serv,channel
0,2022-07-01,T644930978638,Alternative
1,2021-03-01,K68685073049356,Alternative
2,2021-03-01,G68827227560004,Alternative
3,2021-03-01,K64784666024594,Alternative
4,2021-06-01,T68631562544549,Alternative


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Соотношение в столбце channel в %
--------------------------------------------------
Alternative   0.74
Office        0.15
Franchise     0.11
Name: channel, dtype: float64
--------------------------------------------------


In [19]:
# Изменим названия столбцов и типы данных, чтобы было удобнее работать с таблицей
service_fraud.columns = ['dt', 'id_serv', 'channel']
service_fraud.dt = pd.to_datetime(service_fraud.dt, format='%Y-%m-%d')

In [20]:
min_date = service_fraud.dt.min()
max_date = service_fraud.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print('Изучаем данные за период {} дней'.format(len(dates)))

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-08-01 00:00:00
Изучаем данные за период 578 дней


Период так же совпадает с 2умя продидущими датафреймами

### Обьединение датафреймов

In [21]:
service_full = service.merge(technology[['dt', 'id_serv', 'combo']], on=['id_serv', 'dt'], how='left')\
                      .merge(service_revenue, on=['id_serv', 'dt'], how='left')\
                      .merge(service_fraud, on=['id_serv', 'dt'], how='left')

In [22]:
service_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1569812 entries, 0 to 1569811
Data columns (total 7 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   dt       1569812 non-null  datetime64[ns]
 1   id_serv  1569812 non-null  object        
 2   segment  1569812 non-null  int32         
 3   sales    1569812 non-null  object        
 4   combo    570766 non-null   float64       
 5   revenue  1569748 non-null  float64       
 6   channel  2700 non-null     object        
dtypes: datetime64[ns](1), float64(2), int32(1), object(3)
memory usage: 89.8+ MB


In [23]:
# Заменим пропущенные значения в столбце combo на 0 
service_full.loc[service_full.combo.isna(), 'combo'] = '0' 
service_full.combo = service_full.combo.astype(int)

In [24]:
service_full.head()

,dt,id_serv,segment,sales,combo,revenue,channel
0,2021-01-01,A64309659750,1,old,0,"1,889.86",NaN
1,2021-01-01,A643510019114,1,old,1,365.72,NaN
2,2021-01-01,A643511179115,1,old,1,390.79,NaN
3,2021-01-01,A644422566295,1,old,0,85.38,NaN
4,2021-01-01,A644424051299,1,old,1,268.94,NaN


##  Ответы на вопросы
### Какую динамику показывают базы клиентов «Technology», «Service» и Combo в частности?

In [25]:
# База клиентов «Technology»
pt_tech = technology[technology.combo==0].groupby('dt', as_index=False)['id_tech'].count()\
                    .rename(columns={'id_tech':'cnt_tech'})
# База клиентов «Combo»
pt_combo = technology[technology.combo==1].groupby('dt', as_index=False)['id_tech'].count()\
                    .rename(columns={'id_tech':'cnt_combo'})
# База клиентов «Service» (не забываем обрезать лишний месяц)
pt_serv = service_full[(service_full.combo==0) & (service_full.dt<='2022-07-01')]\
                    .groupby('dt', as_index=False)['id_serv'].count()\
                    .rename(columns={'id_serv':'cnt_serv'})

# Итоговая сводная таблица
pt_qu1 = pt_serv.merge(pt_combo, on='dt', how='left').merge(pt_tech, on='dt', how='left') 
pt_qu1

,dt,cnt_serv,cnt_combo,cnt_tech
0,2021-01-01,50152,29062,22250
1,2021-02-01,49673,29152,22134
2,2021-03-01,50279,29281,22211
3,2021-04-01,50115,29943,22033
4,2021-05-01,50773,30182,21705
5,2021-06-01,50607,30466,21252
6,2021-07-01,50817,30783,20978
7,2021-08-01,51221,30784,20965
8,2021-09-01,51246,31415,21162
9,2021-10-01,50385,31695,21253


### Как абоненты Combo ведут себя с точки зрения продукта «Service»? Насколько стабильно пользуются и сколько за «Service» платят? 

In [26]:
# Произзводим группировку по необходимым признакам
pt_qu24 = service_full.pivot_table(index=['dt', 'segment', 'sales', 'combo'], 
                                  aggfunc={'id_serv':'count', 'revenue':['mean', 'sum']}).reset_index()

pt_qu24.columns = ['dt', 'segment', 'sales', 'combo', 'cnt_id', 'mean_rev', 'sum_rev']

In [27]:
pt_qu24

,dt,segment,sales,combo,cnt_id,mean_rev,sum_rev
0,2021-01-01,1,old,0,31870,329.34,"10,495,826.54"
1,2021-01-01,1,old,1,24239,409.48,"9,925,421.58"
2,2021-01-01,2,old,0,14361,117.55,"1,687,733.34"
3,2021-01-01,2,old,1,2863,288.90,"827,120.93"
4,2021-01-01,3,old,0,1974,168.84,"333,281.34"
...,...,...,...,...,...,...,...
169,2022-08-01,1,old,0,33210,352.10,"11,692,514.87"
170,2022-08-01,2,old,0,13132,124.15,"1,630,128.69"
171,2022-08-01,2,sales,0,2,8.74,17.49
172,2022-08-01,3,old,0,1396,219.09,"305,852.22"


### Динамика Combo Продаж. Есть ли среди абонентов FMC продаж – Фрод? Источник фрода?

In [28]:
# Обрезаем лишние строки
frod = service_full[service_full.channel.notna()]
# Произзводим группировку по необходимым признакам
pt_qu3 = frod.pivot_table(index=['dt', 'channel', 'combo'], 
                          aggfunc={'id_serv':'count', 'revenue': 'sum'})\
                          .reset_index().rename(columns={'id_serv':'cnt_id', 'revenue': 'sum_rev'})

In [29]:
pt_qu3

,dt,channel,combo,cnt_id,sum_rev
0,2021-01-01,Alternative,0,74,"12,599.41"
1,2021-01-01,Alternative,1,5,822.99
2,2021-01-01,Franchise,0,26,"4,505.36"
3,2021-01-01,Franchise,1,3,470.92
4,2021-01-01,Office,0,21,"8,523.70"
...,...,...,...,...,...
103,2022-07-01,Office,0,9,"2,192.67"
104,2022-07-01,Office,1,3,997.71
105,2022-08-01,Alternative,0,80,"11,715.84"
106,2022-08-01,Franchise,0,15,"2,602.15"


### Исследовать качество продаж Combo: 
  - Как подключенные клиенты выживают (остаются в базе клиентов) на 1- 4Месяц, 12Месяц с точки зрения продукта «Service»? С точки зрения «Technology»?

**Произвести данный сравнительный анализ можно с помощью 1 и 3 сводных таблиц.**

## SQL Скрипты

В написании скриптов использовал уже измененные названия колонок. Для удобства буду выводить первую строку используемых в скрипте таблиц.

### Какую динамику показывают базы клиентов «Technology», «Service» и Combo в частности?

``` SQL
WITH t1 AS (
SELECT date_trunc('month', "dt")::date AS dt_month,
       count("id_tech") AS cnt_tech
FROM technology
WHERE combo != 1
GROUP BY dt_month),

t2 AS (
SELECT date_trunc('month', "dt")::date AS dt_month,
       count("id_tech") AS cnt_combo
FROM technology
WHERE combo = 1
GROUP BY dt_month),

t3 AS (
SELECT date_trunc('month', s.dt)::date AS dt_month,
       count(s.id_serv) AS cnt_serv
FROM service s 
LEFT JOIN technology t ON (s.id_serv=t.id_serv AND s.dt=t.dt)
WHERE s.dt < '2022-08-01' AND combo != 1
GROUP BY dt_month)


SELECT t1.dt_month, cnt_tech, cnt_combo, cnt_serv
FROM t1
JOIN t2 ON t1.dt_month=t2.dt_month
JOIN t3 ON t1.dt_month=t3.dt_month;

```

In [30]:
# Используемые таблицы
display(technology.head(1))
display(service.head(1))

,dt,id_tech,combo,id_serv
0,2022-02-01,A694897209,1,A64955227227421


,dt,id_serv,segment,sales
0,2021-01-01,A64309659750,1,old


In [31]:
# Итоговая сводная таблица по результатам запроса
pt_qu1

,dt,cnt_serv,cnt_combo,cnt_tech
0,2021-01-01,50152,29062,22250
1,2021-02-01,49673,29152,22134
2,2021-03-01,50279,29281,22211
3,2021-04-01,50115,29943,22033
4,2021-05-01,50773,30182,21705
5,2021-06-01,50607,30466,21252
6,2021-07-01,50817,30783,20978
7,2021-08-01,51221,30784,20965
8,2021-09-01,51246,31415,21162
9,2021-10-01,50385,31695,21253


### Как абоненты Combo ведут себя с точки зрения продукта «Service»? Насколько стабильно пользуются и сколько за «Service» платят? 

```SQL

SELECT date_trunc('month', s.dt)::date AS dt_month,
       s.segment,
       s.sales,
       t.combo,
       count(s.id_serv) AS cnt_serv,
       avg(r.revenue) AS mean_rev,
       sum(r.revenue) AS sum_rev
FROM service s 
LEFT JOIN technology t ON (s.id_serv=t.id_serv AND s.dt=t.dt) 
LEFT JOIN service_revenue r ON (s.id_serv=r.id_serv AND s.dt=r.dt)
GROUP BY dt_month, segment, sales, combo
```

In [32]:
# Используемые таблицы
display(technology.head(1))
display(service.head(1))
display(service_revenue.head(1))

,dt,id_tech,combo,id_serv
0,2022-02-01,A694897209,1,A64955227227421


,dt,id_serv,segment,sales
0,2021-01-01,A64309659750,1,old


,dt,id_serv,revenue
0,2021-04-01,V644328120231,382.72


In [33]:
# Итоговая сводная таблица по результатам запроса
pt_qu24

,dt,segment,sales,combo,cnt_id,mean_rev,sum_rev
0,2021-01-01,1,old,0,31870,329.34,"10,495,826.54"
1,2021-01-01,1,old,1,24239,409.48,"9,925,421.58"
2,2021-01-01,2,old,0,14361,117.55,"1,687,733.34"
3,2021-01-01,2,old,1,2863,288.90,"827,120.93"
4,2021-01-01,3,old,0,1974,168.84,"333,281.34"
...,...,...,...,...,...,...,...
169,2022-08-01,1,old,0,33210,352.10,"11,692,514.87"
170,2022-08-01,2,old,0,13132,124.15,"1,630,128.69"
171,2022-08-01,2,sales,0,2,8.74,17.49
172,2022-08-01,3,old,0,1396,219.09,"305,852.22"


### Динамика Combo Продаж. Есть ли среди абонентов FMC продаж – Фрод? Источник фрода?

Динамику Combo продаж можно вычислить с помощью предидущей таблицы.

```SQL
SELECT date_trunc('month', s.dt)::date AS dt_month,
       t.combo,
       f.channel,
       count(r.id_serv) AS cnt_serv,
       sum(r.revenue) AS sum_rev
FROM service_revenue r
INNER JOIN service_fraud f ON (r.id_serv=f.id_serv AND r.dt=f.dt)
LEFT JOIN technology t ON (r.id_serv=t.id_serv AND r.dt=t.dt) 
GROUP BY dt_month, t.combo, f.channel
```

In [34]:
# Используемые таблицы
display(technology.head(1))
display(service_fraud.head(1))
display(service_revenue.head(1))

,dt,id_tech,combo,id_serv
0,2022-02-01,A694897209,1,A64955227227421


,dt,id_serv,channel
0,2022-07-01,T644930978638,Alternative


,dt,id_serv,revenue
0,2021-04-01,V644328120231,382.72


In [35]:
# Итоговая сводная таблица по результатам запроса
pt_qu3

,dt,channel,combo,cnt_id,sum_rev
0,2021-01-01,Alternative,0,74,"12,599.41"
1,2021-01-01,Alternative,1,5,822.99
2,2021-01-01,Franchise,0,26,"4,505.36"
3,2021-01-01,Franchise,1,3,470.92
4,2021-01-01,Office,0,21,"8,523.70"
...,...,...,...,...,...
103,2022-07-01,Office,0,9,"2,192.67"
104,2022-07-01,Office,1,3,997.71
105,2022-08-01,Alternative,0,80,"11,715.84"
106,2022-08-01,Franchise,0,15,"2,602.15"
